In [1]:
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open("names.txt", "r").read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
block_size = 3
X, Y = [], []

for w in words[:5]:
  print(w)
  context = [0] * block_size # [0, 0, 0] if block_size = 3
  for ch in w + '.':
    ix = stoi[ch] 
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), "--->", ch)
    context = context[1:] + [ix] # Shift the context 1 to the right (context[1:]) and add the next context (ix)
    # This is a window of size of block_size and shifts by 1 to the right every iterations

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [7]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2 ), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn((27), generator=g)
parameters = [C, W1, b1, W2, b2]

In [8]:
print("Total number of parameters:", sum(p.nelement() for p in parameters))

Total number of parameters: 3481


In [9]:
# torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1).shape 
# torch.cat(torch.unbind(emb, dim=1), dim=1).shape Unbind is the same as the above code list (returns a list through dim 1)
# emb.view((36, 6))
# emb.view is efficient. torch.cat is less efficient and uses more memory.

In [11]:
emb = C[X]
emb.shape
h = torch.tanh(emb.view((-1, 6)) @ W1 + b1)
h.shape
logits = h @ W2 + b2
# Cross Entropy Loss is the same as negatie log likelihood
loss = F.cross_entropy(logits, Y)